# Maxfilter analysis

Using the June 2016-versions of `stormdb`-functionality

## Preliminaries
### On the naming logic for the study

In the first round of analysis, the responses in blocks `vis`, `aud`, `green`, `red`, `yellow` and `blue` are to be compared at the sensor level. Therefore, we will use Maxfilter to compensate for within- and between-block differences in head position. If source space analysis is performed at a later stage, the between-block part can (and probably should) be omitted: there movement compensation to the initial head position in each block is probably most relevant.

### Output file and folder names

Remember to use the project `scratch` folder for output, and make it easy to clean up!

In [1]:
from os.path import join
proj_name = 'MINDLAB2014_MEG-Visual-Modulation-MMNm'
scratch_folder = join('/projects', proj_name, 'scratch')
mf_folder = join(scratch_folder, 'maxfilter')  # for maxfilter output
scripts_folder = join('/projects', proj_name, 'scripts')
misc_folder = join('/projects', proj_name, 'misc')
trans_folder = join(scratch_folder, 'trans')  # for transforms

### Use a development version of `mne-python` and `stormdb-python`

This way the version of the analysis code can be kept with the scripts using them, for future reproduction of results. To get the code, and to make a "snapshot" of the state of the code (in case you modify anything accidentally), open a terminal in a remote desktop and (suggestion):

```
cd /projects/MINDLAB2014_MEG-Visual-Modulation-MMNm/misc
git clone https://github.com/mne-tools/mne-python.git
cd mne-python
git checkout -b snapshot_20160630

cd /projects/MINDLAB2014_MEG-Visual-Modulation-MMNm/misc
git clone https://github.com/meeg-cfin/stormdb-python.git
cd stormdb-python
git fetch origing refactor_submit:refactor_submit
git checkout -b refactor_submit
```

In [2]:
# Modify path by adding the mne-python folder from scripts to the beginning.
import sys
sys.path = [join(misc_folder, 'mne-python'),
            join(misc_folder, 'stormdb-python')] + sys.path

### Load libraries

In Python, we have to load what we need!

In [3]:
from stormdb.access import Query
from stormdb.process import Maxfilter
from mne.io import Raw
from mne.bem import fit_sphere_to_headshape
from mne.transforms import rotation_angles, rotation3d, write_trans
import warnings
import os
import re
import numpy as np

In [4]:
# silence mne a bit
from mne.utils import set_log_level
set_log_level('ERROR')

### Constant parameters

Place parameters here you might want to play with, such as tSSS buffer length and correlation limit. Output folders will be automatically generated to reflect these.

In [5]:
tsss_buffer_len = 16
tsss_corr_lim = 0.96
# if you know that some channels are bad or flat, enter them here
# in the form ['2511', '2241']
static_bad_chans = ['1712', '2342'] 

### Finding the data

Instead of accessing raw files directly, use the database query functions to get to files.

In [6]:
qr = Query(proj_name)
subs = qr.get_subjects()


In [7]:
for ii, ss in enumerate(subs):
    print('{0}: {1}'.format(ii, ss))

0: 0007_UVR
1: 0008_LWW
2: 0009_HL8
3: 0010_YEV
4: 0011_HQT
5: 0012_XGT
6: 0013_TT3
7: 0014_JRC
8: 0015_PGT
9: 0016_WNJ
10: 0017_H3P
11: 0018_CDT
12: 0019_AWE
13: 0020_YMG
14: 0021_MIS
15: 0022_WZE
16: 0023_FS9
17: 0024_WU1
18: 0025_F9E
19: 0026_XYD
20: 0028_OZZ
21: 0029_XMB
22: 0030_HT5
23: 0031_HOF
24: 0032_28N
25: 0033_ACE
26: 0034_O1V
27: 0035_J6E
28: 0036_DWP
29: 0037_PPO
30: 0038_IC8
31: 0039_WMY
32: 0040_ODM
33: 0041_NDF
34: 0042_RN0
35: 0043_KQC
36: 0044_F8Y
37: 0045_MEX
38: 0046_JYT
39: 0047_1EY
40: 0048_ESO
41: 0049_NIG
42: 0050_L2X
43: 0051_HJY
44: 0052_CTW
45: 0053_BIV
46: 0054_JMT
47: 0055_AP4
48: 0056_2H3


In [8]:
len(subs)

49

## Select subject

Instead of writing a script that loops over subjects (could be done), this notebook requires you to select one subject at a time. The advantage is that you then sanity-check each stage before continuing. The final `submit_to_isis` commands do not block the notebook: you can immediately go forward with the next subject. New submissions will simply be added to the queue.

To make this semi-automatic, the variable `cur_sub_index` will be incremented by 1 every time the `submit`-command is issued at the end of this notebook. You can also just manually set the index to what you want.

In [9]:
cur_sub_index = 0  # see below for the meaning of this
sub_specific_bad_chans = []  # empty list if no (more) bad chans

In [66]:
cur_sub = subs[cur_sub_index]
print('Current subject: {sub:s}'.format(sub=cur_sub))

Current subject: 0007_UVR


In [ ]:
# cur_sub = '0008'

## Calculating the head positions

Here we want to calculate the average initial head position and use movecomp to correct head motion to that origin.

In [12]:
description = '*vis|*aud|*green|*red|*yellow|*blue'
DATAblocks = qr.filter_series(description=description, subjects=cur_sub, modalities='MEG')

if len(DATAblocks) != 6:
    raise RuntimeError('Not all 6 blocks found for {0}, please check!'.format(cur_sub))
for ib in range(len(DATAblocks)):
    print('{:d}: {:s}'.format(ib + 1, DATAblocks[ib]['path']))

1: /projects/MINDLAB2014_MEG-Visual-Modulation-MMNm/raw/0007/20140623_000000/MEG/002.MEG_MMN_green/files
2: /projects/MINDLAB2014_MEG-Visual-Modulation-MMNm/raw/0007/20140623_000000/MEG/003.MEG_MMN_red/files
3: /projects/MINDLAB2014_MEG-Visual-Modulation-MMNm/raw/0007/20140623_000000/MEG/004.MEG_MMN_vis/files
4: /projects/MINDLAB2014_MEG-Visual-Modulation-MMNm/raw/0007/20140623_000000/MEG/005.MEG_MMN_blue/files
5: /projects/MINDLAB2014_MEG-Visual-Modulation-MMNm/raw/0007/20140623_000000/MEG/006.MEG_MMN_aud/files
6: /projects/MINDLAB2014_MEG-Visual-Modulation-MMNm/raw/0007/20140623_000000/MEG/007.MEG_MMN_yellow/files


### Find initial head positions

Get device to head-transformation from the initial HPI fit at the beginning of each acquisition. This consists of a translation and rotation, which are combined in to a single transformation matrix. Since both operations are *affine transformations*, we may simply average the initial matrices to obtain the mean head position and rotation.

In [13]:
init_xfm = []
init_rot = []
for bl in DATAblocks:
    fname = join(bl['path'], bl['files'][0])  # first file is enough
    with warnings.catch_warnings():  # suppress some annoying warnings for now
        warnings.simplefilter("ignore")
        info = Raw(fname, preload=False, verbose=False).info

    init_xfm += [info['dev_head_t']['trans']]
    # translations: info['dev_head_t']['trans'][:, 3][:-1]
    init_rot += [info['dev_head_t']['trans'][:3, :3]]

This filename (/raw/sorted/MINDLAB2014_MEG-Visual-Modulation-MMNm/0007/20140623_000000/MEG/002.MEG_MMN_green/files/PROJ0153_SUBJ0007_SER002_FILESNO001.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
This filename (/raw/sorted/MINDLAB2014_MEG-Visual-Modulation-MMNm/0007/20140623_000000/MEG/003.MEG_MMN_red/files/PROJ0153_SUBJ0007_SER003_FILESNO001.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
This filename (/raw/sorted/MINDLAB2014_MEG-Visual-Modulation-MMNm/0007/20140623_000000/MEG/004.MEG_MMN_vis/files/PROJ0153_SUBJ0007_SER004_FILESNO001.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
This filename (/raw/sorted/MINDLAB2014_MEG-Visual-Modulation-MMNm/00

Find the average head position and calculate how far each block is from it (look for outliers).

In [14]:
mean_init_xfm = np.mean(np.stack(init_xfm), axis=0)  # stack, then average over new dim
init_rot_angles = [rotation_angles(m) for m in init_rot]

mean_init_rot_xfm = rotation3d(*tuple(np.mean(np.stack(init_rot_angles),
                                              axis=0)))  # stack, then average, then make new xfm

assert(np.sum(mean_init_xfm[-1]) == 1.0)  # sanity check result
mean_trans = info['dev_head_t']  # use the last info as a template
mean_trans['trans'] = mean_init_xfm  # replace the transformation
mean_trans['trans'][:3, :3] = mean_init_rot_xfm  # replace the rotation part

Plot the mean position in mm and each block's discrepancy from the mean.

In [15]:
mean_init_headpos = mean_trans['trans'][:-1, -1]  # meters
print('Mean head position (device coords): ({:.1f}, {:.1f}, {:.1f}) mm'.\
      format(*tuple(mean_init_headpos*1e3)))
print('Block discrepancies from mean:')
for ib, xfm in enumerate(init_xfm):
    diff = 1e3 * (xfm[:-1, -1] - mean_init_headpos)
    rmsdiff = np.linalg.norm(diff)
    print('\tblock {:d}: norm {:.1f} mm ({:.1f}, {:.1f}, {:.1f}) mm '.\
          format(ib + 1, rmsdiff, *tuple(diff)))

Mean head position (device coords): (3.8, -10.7, 42.2) mm
Block discrepancies from mean:
	block 1: norm 8.2 mm (-0.3, -8.1, -1.2) mm 
	block 2: norm 6.4 mm (0.5, 6.4, 0.3) mm 
	block 3: norm 5.2 mm (0.6, 5.2, 0.2) mm 
	block 4: norm 15.0 mm (-1.3, -14.7, -2.6) mm 
	block 5: norm 5.3 mm (0.3, 5.1, 1.3) mm 
	block 6: norm 6.4 mm (0.1, 6.1, 2.0) mm 


Do the same for rotations

In [16]:
mean_rots = rotation_angles(mean_trans['trans'][:3, :3])  # these are in radians
mean_rots_deg = tuple([180. * rot / np.pi for rot in mean_rots])  # convert to deg
print('Mean head rotations (around x, y & z axes): ({:.1f}, {:.1f}, {:.1f}) deg'.\
      format(*mean_rots_deg))
print('Block discrepancies from mean:')
for ib, rot in enumerate(init_rot):   
    cur_rots = rotation_angles(rot)
    diff = tuple([180. * cr / np.pi - mr for cr, mr in zip(cur_rots, mean_rots_deg)])
    print('\tblock {:d}: ({:.1f}, {:.1f}, {:.1f}) deg '.\
          format(ib + 1, *tuple(diff)))

Mean head rotations (around x, y & z axes): (6.6, -1.5, 1.0) deg
Block discrepancies from mean:
	block 1: (-0.4, -0.2, -0.0) deg 
	block 2: (-6.0, 1.5, -1.2) deg 
	block 3: (0.1, 0.7, 1.2) deg 
	block 4: (0.9, -0.5, -0.1) deg 
	block 5: (4.1, -0.1, -0.6) deg 
	block 6: (1.3, -1.4, 0.6) deg 


Save the new mean transformation to a file to be used later in the `maxfilter`-option `trans`.

In [17]:
mean_trans_folder = join(trans_folder, cur_sub)
if not os.path.exists(mean_trans_folder):
    os.makedirs(mean_trans_folder)
mean_trans_file = join(mean_trans_folder, 'allblocks_mean-trans.fif')
write_trans(mean_trans_file, mean_trans)

## Fit head origin for SSS expansion

any info (from this study) will do, since the digitization points are the same for all blocks; take the last one from for-loop above. NB: Only use EEG locations, since head points only on face!

In [18]:
set_log_level('INFO')
rad, origin_head, ori_dev = fit_sphere_to_headshape(info,
                                                    dig_kinds='extra',
                                                    units='mm')
set_log_level('ERROR')

Fitted sphere radius:         85.8 mm
Origin head coordinates:      -7.2 4.7 41.9 mm
Origin device coordinates:    -10.8 15.5 -1.9 mm


## Initiate Maxfilter-object

In [40]:
mf = Maxfilter(proj_name, bad=static_bad_chans + sub_specific_bad_chans)

### Build maxfilter commands for all the blocks

First set some of the options (leave others as default).

In [41]:
mfopts = dict(
    origin = '{:.1f} {:.1f} {:.1f}'.format(*tuple(origin_head)),  # mm
    frame = 'head',
    force = True,  # overwrite if needed
    autobad = 'on',  # or use xscan first
    st = True,  # use tSSS
    st_buflen = tsss_buffer_len,  # parameter set in beg. of notebook
    st_corr = tsss_corr_lim,  # parameter set in beg. of notebook
    movecomp = True,
    trans = mean_trans_file,  # compensate to mean initial head position (saved to file),
                              # use None for initial head position
    logfile = None,  # we replace this in each loop
    hp = None,  # head positions, replace in each loop
    n_threads = 4  # antal kerner på isis, max 12, være solidarisk!
    )

mne-python likes raw and raw-like (tsss) files that are part of a long (>2GB) continuous acquisition to follow the convention:

1. `filename_raw_tsss.fif` (first file)
1. `filename_raw_tsss-1.fif` (second file, etc.)


In [42]:
out_folder = join(mf_folder,
                  'tsss_st{:d}_corr{:.0f}'.format(mfopts['st_buflen'],
                                                  np.round(100 * mfopts['st_corr'])),
                  cur_sub)

# Check that output path exists
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [43]:
for blockno, bl in enumerate(DATAblocks):
    for fileno, fil in enumerate(bl['files']):
        in_fname = join(bl['path'], bl['files'][fileno])
        
        series_name = re.search('(vis|aud|green|red|yellow|blue)',
                                bl['seriename']).group(1)
        
        out_fname = join(out_folder, '{0}_raw_tsss.fif'.format(series_name))
        if fileno > 0:
            out_fname = out_fname[:-4] + '-{:d}.fif'.format(fileno)
           
        mfopts['logfile'] = out_fname[:-3] + 'log'
        mfopts['hp'] = out_fname[:-3] + 'pos'
        
        mf.build_cmd(in_fname, out_fname, **mfopts)

## Submit to isis for processing

First check that you think sane things will happen, if you like.

In [44]:
# mf.print_input_output_mapping()

If in doubt, uncomment this line to see the actual commands that will execute:

In [45]:
# mf.commands

### Optional: What is the cluster doing?

In [30]:
# mf.cluster.get_load()

### Ready to rock

In [46]:
mf.submit()

Cluster job submitted, job ID: 3511710
Cluster job submitted, job ID: 3511711
Cluster job submitted, job ID: 3511712
Cluster job submitted, job ID: 3511713
Cluster job submitted, job ID: 3511714
Cluster job submitted, job ID: 3511715


### Checking what's going on

In a terminal:

```
qstat
```

shows all running jobs (in your name). For _every/anyone's_ jobs, run

```
qstat -u "*"
```

In [61]:
mf.status

#1 (3511710): Waiting in the queue
#2 (3511711): Waiting in the queue
#3 (3511712): Waiting in the queue
#4 (3511713): Waiting in the queue
#5 (3511714): Waiting in the queue
#6 (3511715): Waiting in the queue


To kill a submitted (or even running) job

In [63]:
# mf.kill(3511711)

Job 3511711 killed. You must manually delete any output it may have created!


To kill all submitted jobs:

In [65]:
# mf.kill()

To kill a job in the shell:

```
qdel JOB_NUMBER
```

or for all jobs (in your name):

```
qdel *
```

In [59]:
try:
    mf_list
except NameError:
    mf_list = []
mf_list += [mf]

In [60]:
for cur_mf in mf_list:
    cur_mf.status

#1 (3511710): Waiting in the queue
#2 (3511711): Waiting in the queue
#3 (3511712): Waiting in the queue
#4 (3511713): Waiting in the queue
#5 (3511714): Waiting in the queue
#6 (3511715): Waiting in the queue
#1 (3511710): Waiting in the queue
#2 (3511711): Waiting in the queue
#3 (3511712): Waiting in the queue
#4 (3511713): Waiting in the queue
#5 (3511714): Waiting in the queue
#6 (3511715): Waiting in the queue
